In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import keras
import pandas as pd
import numpy as np
import cv2
import os
from random import shuffle
from tqdm import tqdm
import scipy
import skimage
from skimage.transform import resize

In [ ]:
train_dir = "/content/drive/My Drive/Colab Notebooks/chest_xray/train/"
test_dir = "/content/drive/My Drive/Colab Notebooks/chest_xray/test/"
val_dir = "/content/drive/My Drive/Colab Notebooks/chest_xray/val/"

In [ ]:
from keras import models
from keras import layers

In [ ]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
                 include_top = False,
                 input_shape = (224, 224, 3))

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dropout(rate=1- 0.7))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dropout(rate = 1-0.7))
model.add(layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model.summary()

In [ ]:
conv_base.trainable = False

In [ ]:
model.summary()

In [ ]:
from keras import optimizers
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
batch_size = 50
IMG_SHAPE = 224

In [ ]:
image_gen = ImageDataGenerator(rescale = 1./255, 
                               horizontal_flip = True,
                              rotation_range=5,
                              width_shift_range= 0.10,
                              height_shift_range= 0.05,
                              zoom_range = 0.15,
                              shear_range = 0.1,
                              vertical_flip = False,
                              fill_mode = 'reflect')

train_data_gen = image_gen.flow_from_directory(batch_size = batch_size,
                                               directory= train_dir,
                                               shuffle=True,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                class_mode='binary'
                                                              )

In [ ]:
image_val_gen = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen.flow_from_directory(batch_size = batch_size,
                                                directory = val_dir,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                class_mode='binary'
                                                              )

In [ ]:
history = model.fit_generator(
                               train_data_gen,
                                steps_per_epoch= 5
                                epochs = 15,
                                 validation_data = val_data_gen,
                                 validation_steps =40)

In [ ]:
import matplotlib.pyplot as plt

epochs =15
acc = history.history['acc']
loss = history.history['loss']

val_acc = history.history['val_acc']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
image_test_gen = ImageDataGenerator(rescale= 1./255)

test_data_gen = image_val_gen.flow_from_directory(batch_size = batch_size,
                                                directory = test_dir,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                class_mode='binary'
                                                              )

In [ ]:
scores = model.evaluate_generator(test_data_gen)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
import numpy
test_steps_per_epoch = numpy.math.ceil(test_data_gen.samples / test_data_gen.batch_size)

predictions = model.predict_generator(test_data_gen, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = numpy.argmax(predictions, axis=1)

##Get ground-truth classes and class-labels

true_classes = test_data_gen.classes
class_labels = list(test_data_gen.class_indices.keys())   


##Use scikit-learn to get statistics
import sklearn.metrics as metrics

report = metrics.confusion_matrix(true_classes, predicted_classes, target_names=class_labels)
print(report) 

In [ ]:
from sklearn.metrics import confusion_matrix
report = confusion_matrix(true_classes, predicted_classes)


In [ ]:
print(report)